In [1]:
# ### basic required packages ###
!pip install numpy
!pip install pyopenms
!pip install biopython
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 MB 6.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 12.0 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 1.6 MB/s eta 0:00:00a 0:00:01


In [2]:
# ### GOOGLE AUTH ###
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip3 install --upgrade oauth2client 
!pip3 install --upgrade oauth2client 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 50.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 173.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 167.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 181.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.1/229.1 kB 184.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 150.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 230.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 117.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 1.2 MB/s eta 0:00:00ta 0:00:01


In [82]:
#######################
###  Запускать тут ####
#######################

import glob
import pandas as pd
import os
import unicodedata

# # CHANGE THIS
# EXPERIMENT_NAME = "KJE0006"

# Список для хранения обработанных DataFrame из каждого файла


def xlsx_paths(EXPERIMENT_NAME):
    # specifying the path to csv files
    path = f"./requests/{EXPERIMENT_NAME}"

    # csv files in the path
    file_paths = glob.glob(path + "/*.xlsx")
    return file_paths

def csv_paths(EXPERIMENT_NAME):
    # specifying the path to csv files
    path = f"./requests/{EXPERIMENT_NAME}"

    txt_filepaths = glob.glob(path + "/*.txt")
    print(txt_filepaths)
    
    for path in txt_filepaths:
        os.replace(path, path.replace(".txt", ".csv"))
    
    # csv files in the path
    file_paths = glob.glob(path + "/*.csv")
    
    return file_paths

def is_valid_sequence(seq):

    if pd.isna(seq):
        print(f"{seq} is invald sequence!")
        return False
    valid_chars = set("ATGCUYRSWKMBDHVN.-")
    return all(c in valid_chars for c in str(seq))

def process_excel_files(xlsx_paths, dfs_list):

    for file_path in xlsx_paths:
        import numpy as np
        # Чтение файла Excel
        df = pd.read_excel(file_path, header=None)
        print(df)

        if len(df.columns)==1:
            df = df[0].str.split(',', expand=True)

        if len(df.columns)<4:
        #     return print("ERROR IN INPUT FILE")
            pass
        df.dropna(how='any', subset=(0,2,3), inplace=True)

        df[1].fillna("DMT OFF", inplace = True)

        # df curation
        for column in df.columns:
            df[column] = df[column].apply(lambda x: unicodedata.normalize('NFC', str(x)))
        #  pandas' Series.str.strip() method
        df = df.apply(lambda x: x.str.strip())

        df[0] = df[0].apply(lambda x: x.upper())

        # Получение имени самого левого столбца
        leftmost_column = df.columns[0]
        print(file_path, len(df))
        # Фильтрация строк согласно заданным условиям
        df = df[df[leftmost_column].apply(is_valid_sequence)]

        print(file_path, len(df))
        # Добавление обработанного DataFrame в список
        dfs_list.append(df)
        
        if dfs_list != None:
            pass
        else:
            dfs_list = []
            print([])
        
    return dfs_list
        
    
def process_csv_files(csv_paths, dfs_list):

    for file_path in csv_paths:
        # Чтение файла csv
        df = pd.read_csv(file_path, header=None, sep=",")
#         print(df)
        df.dropna(how='any', inplace=True)
        if len(df.columns)==1:
            df = df[0].str.split(',', expand=True)
            
        if len(df.columns)==1:
            df = df[0].str.split(';', expand=True)

        if len(df.columns)<4:
            return print("ERROR IN INPUT FILE")

        # df curation
        for column in df.columns:
            df[column] = df[column].apply(lambda x: unicodedata.normalize('NFC', str(x)))
        #  pandas' Series.str.strip() method
        df = df.apply(lambda x: x.str.strip())
        
        df[0] = df[0].apply(lambda x: x.upper())

        # Получение имени самого левого столбца
        leftmost_column = df.columns[0]
        print(file_path, len(df))
        # Фильтрация строк согласно заданным условиям
        df = df[df[leftmost_column].apply(is_valid_sequence)]
        
        # Добавление обработанного DataFrame в список
        dfs_list.append(df)
        
        if dfs_list != None:
            pass
        else:
            dfs_list = []
            print([])
            
    return dfs_list
        
def process_files(EXPERIMENT_NAME):
    dfs_list = []
    dfs_list = process_excel_files(xlsx_paths(EXPERIMENT_NAME), dfs_list)
    if dfs_list != None:
        print(dfs_list)
    else:
        dfs_list = []
    dfs_list = process_csv_files(csv_paths(EXPERIMENT_NAME), dfs_list)
    return dfs_list

# EXPERIMENT_NAME = input()
# dfs_list=process_files(EXPERIMENT_NAME)



In [83]:
EXPERIMENT_NAME = "KJE0035" # input()
# dfs_list=process_csv_files(csv_paths(EXPERIMENT_NAME), dfs_list)

# xlsx_paths(EXPERIMENT_NAME)
# csv_paths(EXPERIMENT_NAME)

# dfs_list = []
# dfs_list = process_csv_files(csv_paths(EXPERIMENT_NAME), dfs_list)

# dfs_list = []
# dfs_list = process_excel_files(xlsx_paths(EXPERIMENT_NAME), dfs_list)

dfs_list = process_files(EXPERIMENT_NAME)


                                                   0                1  \
0  Для создания заказа сначала создайте копию дан...              NaN   
1  Введите последовательность олигонуклеотида тол...              NaN   
2                                 ПОСЛЕДОВАТЕЛЬНОСТЬ  ЭТО НЕ ИЗМЕНЯТЬ   
3                           TTAARTTACAAAAAAACATGGGAC           DMT ON   
4                            AAAGAATATCTTTTCCTTCAGGG           DMT ON   
5                             ATGGGACAAGTGAAAATGTCTC           DMT ON   
6                           AATTACTCATAATCATTTTGACTG           DMT ON   

                                                   2  \
0                                                NaN   
1  Введите от 3 до 14 знаков, используя латинские...   
2                               ШИФР ОЛИГОНУКЛЕОТИДА   
3                                               N-oF   
4                                               N-oR   
5                                               N-iF   
6                      

                                                   0                1  \
0  Для создания заказа сначала создайте копию дан...              NaN   
1  Введите последовательность олигонуклеотида тол...              NaN   
2                                 ПОСЛЕДОВАТЕЛЬНОСТЬ  ЭТО НЕ ИЗМЕНЯТЬ   
3         AGAAAGATCTCGAGCTCAAGCTATGGTTTCACCGGCATCTGC           DMT ON   
4   CGGTGAAGAAAGATCTCGAGCTCAAGCTATGGTTAATAACAACAACCG           DMT ON   
5   GGTACCGTCGACTGCAGAATTCGAAGCTTTATAAACCGAGTCCATTGA           DMT ON   
6                                  AGCTTCGAATTCTGCAG           DMT ON   
7   CGGTGAAGAAAGATCTCGAGCTCAAGCTAACAAAACAAGTTCAATGGT           DMT ON   
8          GGTACCGTCGACTGCAGAATTCGAAGCTTCGCGCTAAGGGG           DMT ON   

                                                   2  \
0                                                NaN   
1  Введите от 3 до 14 знаков, используя латинские...   
2                               ШИФР ОЛИГОНУКЛЕОТИДА   
3                                           T

In [81]:
file_path = xlsx_paths(EXPERIMENT_NAME)[0]
import numpy as np
# Чтение файла Excel
df = pd.read_excel(file_path, header=None)
# print(df)

if len(df.columns)==1:
    df = df[0].str.split(',', expand=True)

if len(df.columns)<4:
#     return print("ERROR IN INPUT FILE")
    pass
df.dropna(how='any', subset=(0,2,3), inplace=True)

df[1].fillna("DMT OFF", inplace = True)

# df curation
for column in df.columns:
    df[column] = df[column].apply(lambda x: unicodedata.normalize('NFC', str(x)))
#  pandas' Series.str.strip() method
df = df.apply(lambda x: x.str.strip())

df[0] = df[0].apply(lambda x: x.upper())

# Получение имени самого левого столбца
leftmost_column = df.columns[0]
print(file_path, len(df))
print(is_valid_sequence(df[leftmost_column][3]))
# Фильтрация строк согласно заданным условиям
df = df[df[leftmost_column].apply(is_valid_sequence)]

print(file_path, len(df))
# Добавление обработанного DataFrame в список
dfs_list.append(df)

if dfs_list != None:
    pass
else:
    dfs_list = []
    print([])
    
df

./requests/KJE0034/ЗАКАЗ NP - Николай Постика.xlsx 3
False
./requests/KJE0034/ЗАКАЗ NP - Николай Постика.xlsx 0


,0,1,2,3


In [86]:
print(len(dfs_list))
for i in dfs_list:
    print(len(i), "\n", i.head(1)[3], "\n\n",)

export_to_input_files(dfs_list, EXPERIMENT_NAME)

22
4 
 3    LUKINA
Name: 3, dtype: object 


4 
 3    VOLOK
Name: 3, dtype: object 


98 
 0    Nikiteev
Name: 3, dtype: object 


10 
 3    SEMASHKO
Name: 3, dtype: object 


4 
 3    SHCHERBAKOVA
Name: 3, dtype: object 


69 
 0    GLK054
Name: 3, dtype: object 


11 
 3    CHUDINOV
Name: 3, dtype: object 


4 
 3    LUKINA
Name: 3, dtype: object 


16 
 0    KUZMINA
Name: 3, dtype: object 


24 
 3    MAKARIKOVA
Name: 3, dtype: object 


1 
 3    POSTIKA
Name: 3, dtype: object 


6 
 3    CHUDINOV
Name: 3, dtype: object 


122 
 3    GLADYSHEVA
Name: 3, dtype: object 


16 
 0    KUZMINA
Name: 3, dtype: object 


18 
 0    FISUNOV
Name: 3, dtype: object 


51 
 0    ZAYNULLIN_TdTcons3
Name: 3, dtype: object 


10 
 6    CUSTOMER
Name: 3, dtype: object 


48 
 0    ZAYNULLIN_TdTcons2
Name: 3, dtype: object 


49 
 0    ZAYNULLIN_TdTcons1
Name: 3, dtype: object 


49 
 0    DIY
Name: 3, dtype: object 


122 
 0    FISUNOV
Name: 3, dtype: object 


18 
 0    FISUNOV
Name: 3, dtype: obj

In [6]:
# # process_excel_files(xlsx_paths(EXPERIMENT_NAME), dfs_list)
# from google.colab import data_table
# data_table.enable_dataframe_formatter()

# from IPython.display import display

# # print the DataFrame
# display(dfs_list[-2][10:])

# Nikiteev_df=dfs_list[-2]

# Nikiteev_df=Nikiteev_df.dropna(how='any', inplace=True)

# display(Nikiteev_df[10:59])


In [15]:

def export_to_input_files(dfs_list, EXPERIMENT_NAME):
    # Конкатенация всех обработанных DataFrame по строкам
    final_df = None
    final_df = pd.concat(dfs_list, axis=0, join='outer', ignore_index=True)
    
    if (len(final_df)>384 and len(final_df)<=768):

        df_A = final_df[:384]
        df_B = final_df[384:]

        # Сохранение результата в новый файл CSV
        df_A.to_csv(f"./result_input/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_concatenated_file_plate_A.csv", index=False, sep = ',', header = None, encoding = "UTF-8")
        # Сохранение результата в новый файл Excel
        df_A.to_excel(f"./result_input/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_concatenated_file_plate_A.xlsx", index=False, header = None, encoding = "UTF-8")

         # Сохранение результата в новый файл CSV
        df_B.to_csv(f"./result_input/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_concatenated_file_plate_B.csv", index=False, sep = ',', header = None, encoding = "UTF-8")
        # Сохранение результата в новый файл Excel
        df_B.to_excel(f"./result_input/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_concatenated_file_plate_B.xlsx", index=False, header = None, encoding = "UTF-8")

    elif len(final_df)>768:
    
        print("error, too much oligos HALP")
    
    else:

        # Сохранение результата в новый файл CSV
        final_df.to_csv(f"./result_input/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_concatenated_file.csv", index=False, sep = ',', header = None, encoding = "UTF-8")

        # Сохранение результата в новый файл Excel
        final_df.to_excel(f"./result_input/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_concatenated_file.xlsx", index=False, header = None, encoding = "UTF-8")

In [49]:
export_to_input_files(dfs_list, EXPERIMENT_NAME)

In [17]:
#!/usr/bin/env python
#
# File: DrOligoInputPreparation_withFormsAPI.py
# Author: Oleg Fedorov, Kamil Zaynullin
#
# Copyright (C) 2023 Group of automated synthesis
#
# This file is part of gene synthesis automatization pipeline implemented in SBM.

from __future__ import print_function

import io
import os

########################
## GOOGLE API IMPORTS ##
########################
from apiclient import discovery
from google.oauth2.credentials import Credentials
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
from httplib2 import Http
from oauth2client import client, file, tools

SCOPES = "https://www.googleapis.com/auth/drive"
DISCOVERY_DOC = "https://forms.googleapis.com/$discovery/rest?version=v1"


def creating_dirs(EXPERIMENT_NAME):
    #################
    ## CREATE DIR  ##
    #################
    cwd = os.getcwd()
    dir = os.path.join(cwd, "requests", EXPERIMENT_NAME)
    if not os.path.exists(dir):
        os.mkdir(dir)
        
    dir = os.path.join(cwd, "result_input", EXPERIMENT_NAME)
    if not os.path.exists(dir):
        os.mkdir(dir)


def google_auth():
    #################
    ## GOOGLE AUTH ##
    #################

    """
    caution! it appeares to be deprecated!!!

    """
    cwd = os.getcwd()
    token_file = os.path.join(cwd, "token.json")
    if os.path.exists(token_file):
        print("auth token already exists")
        store = file.Storage(token_file)
        creds = store.get()
    else:
        store = file.Storage("token.json")
        creds = None
        if not creds:
            flow = client.flow_from_clientsecrets("credentials.json", SCOPES)

            creds = tools.run_flow(flow, store)
    return creds


def retrive_forms_data(creds):
    ##################
    ## GOOGLE FORMS ##
    ##################
    service = discovery.build(
        "forms",
        "v1",
        http=creds.authorize(Http()),
        discoveryServiceUrl=DISCOVERY_DOC,
        static_discovery=False,
    )

    form_id = "1bVGyD-pz5w3BXM48yHWnhEDVnYNVgzRrF3ysvjAbqHQ"
    result = service.forms().responses().list(formId=form_id).execute()
    file_list = [
        response["answers"]["778e4a4f"]["fileUploadAnswers"]["answers"]
        for response in result["responses"]
    ]
    print(file_list)
    return file_list


####################################################
# downloading from Google Disk and writing to file #
####################################################
def download_request_file(fileID, filename, creds, EXPERIMENT_NAME):
    try:
        service = discovery.build("drive", "v3", credentials=creds)

        file_id = fileID

        # pylint: disable=maybe-no-member
        request = service.files().get_media(fileId=file_id)
        file = io.BytesIO()
        downloader = MediaIoBaseDownload(file, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f"Download {int(status.progress() * 100)}.")

    except HttpError as error:
        print(f"An error occurred: {error}")
        file = None
    filepath = os.path.join(os.getcwd(), "requests", EXPERIMENT_NAME, filename)
    with open(filepath, "wb") as outfile:
        # Copy the BytesIO stream to the output file
        outfile.write(file.getbuffer())

    return print(file.getvalue())


def download_all_files(creds, EXPERIMENT_NAME):
    for file in retrive_forms_data(creds):
        download_request_file(file[0]["fileId"], file[0]["fileName"], creds, EXPERIMENT_NAME)


##########
## main ##
##########
def google_main(): # -> None:
    EXPERIMENT_NAME = input("enter EXPERIMENT_NAME:")
    creating_dirs(EXPERIMENT_NAME)
    # retrive_forms_data()
    creds = google_auth()
    download_all_files(creds, EXPERIMENT_NAME)
    print("downloads finished")
    return EXPERIMENT_NAME

In [18]:
#####################################
#### EXPORTING EMAILS FROM FORMS ####
#####################################

import re  
import json

def get_emails_data():
    creds = google_auth()

    service = discovery.build(
        "forms",
        "v1",
        http=creds.authorize(Http()),
        discoveryServiceUrl=DISCOVERY_DOC,
        static_discovery=False,
    )

    form_id = "1bVGyD-pz5w3BXM48yHWnhEDVnYNVgzRrF3ysvjAbqHQ"
    result = service.forms().responses().list(formId=form_id).execute()
#     print(json.dumps(result, indent=2))
    
    return result

def remove_duplicate_emails(response_str):
    response_dict = json.loads(response_str) # Преобразование JSON-строки в Python-объект
    email_set = set()

    for response in response_dict['responses']:
        email = response.get('respondentEmail', None)
        if email:
            email_set.add(email)

        email_set.add(response['answers']['4d5a72c8']['textAnswers']['answers'][0]['value'])

    email_list = list(email_set)

#     print(email_list)
    return email_list

regex = re.compile(r'([A-Za-z0-9]+[.-_])*[A-Za-z0-9\-]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+')  
  
def emailValid(unique_emails):
    iterable = unique_emails.copy()
    for item in iterable:
#         print(item)
        if re.fullmatch(regex, item):
            pass
            print(item, "The given mail is valid")  
        else:
            unique_emails.remove(item)
            print(item, "The given mail is invalid")  
#     print(len(iterable))
    return unique_emails

def create_file_with_emails(EXPERIMENT_NAME):
    # Использования функции
    result = get_emails_data()
    response_str = json.dumps(result, indent=2)
    unique_emails = remove_duplicate_emails(response_str)
#     print(unique_emails)
    unique_emails = emailValid(unique_emails)
#     print(unique_emails)
    
    # write to file
    with open(f"./result_input/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_customer_emails.txt", 'w') as f:
        for email in unique_emails:
            f.write(f"{email}\n")


In [19]:
EXPERIMENT_NAME = "KJE0035"
def combined_pipeline():
    EXPERIMENT_NAME = google_main()
    print(EXPERIMENT_NAME)
    dfs_list=process_files(EXPERIMENT_NAME)
    print(len(dfs_list))
    export_to_input_files(dfs_list, EXPERIMENT_NAME)
    create_file_with_emails(EXPERIMENT_NAME)
    return dfs_list
dfs_list = combined_pipeline()

enter EXPERIMENT_NAME:KJE0035
auth token already exists
[[{'fileId': '1XTId5D1ARBGxAUS48SIEJJGnKZc_k_8Y', 'fileName': 'Заказ олиги Волок_22052024 - Viktor P.V..xlsx', 'mimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'}], [{'fileId': '1koSVRsv1xWOUGwotRWRMHqYViKUV9nha', 'fileName': 'Калибровочные буквы - Юлия Кузьмина.xlsx', 'mimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'}], [{'fileId': '1h0omJWs3v32HThai33vEnzIHm2ft_6aQ', 'fileName': 'ФОРМА ЗАКАЗА_Семашко_2024_05_15 - Tatiana Semashko.xlsx', 'mimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'}], [{'fileId': '1V5wjKrtr9W465d48-1ymsvwFtj_A_WRm', 'fileName': 'Олиги - корректированный гликолиз и часть ПФП - Глеб Фисунов.csv', 'mimeType': 'application/vnd.ms-excel'}], [{'fileId': '15ShcLDZMdIJeDqvLnu0FuGTXv07nwT1y', 'fileName': '240522 Chudinov request - Ivan Chudinov.xlsx', 'mimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'

Download 100.
b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00b\xee\x9dh^\x01\x00\x00\x90\x04\x00\x00\x13\x00\x08\x02[Content_Types].xml \xa2\x04\x02(\xa0\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

Download 100.
b'PK\x03\x04\x14\x00\x08\x08\x08\x00\x025\xafX\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18\x00\x00\x00xl/drawings/drawing1.xml\x9d\xd0]n\xc20\x0c\x07\xf0\x13\xec\x0eU\xdeiZ\x18\x13C\x14^\xd0N0\x0e\xe0%n\x1b\x91\x8f\xca\x0e\xa3\xdc~\xd1J6i{\x01\x1em\xcb?\xf9\xef\xcdnt\xb6\xf8Db\x13|#\xea\xb2\x12\x05z\x15\xb4\xf1]#\x0e\xefo\xb3\x95(8\x82\xd7`\x83\xc7F\\\x90\xc5n\xfb\xb4\x195\xad\xcf\xbc\xa7"\xed{^\xa7\xb2\x11}\x8c\xc3ZJV=:\xe02\x0c\xe8\xd3\xb4\r\xe4 \xa6\x92:\xa9\t\xceIvV\xce\xab\xeaE\xf2@\x08\x9a{\xc4\xb8\x9f&\xe2\xea\xc1\x03\x9a\x03\xe3\xf3\xfeM\xd7\x84\xb65\n\xf7A\x9d\x1c\xfa8!\x84\x16b\xfa\x05\xf7f\xe0\xac\xa9\x07\xaeQ=P\xfc\x01\xc6\x7f\x823\x8a\x02\x876\x96*\xb8\xeb)\xd9HB\xfd<\t8\xfe\x1a\xf5\xdd\xc8R\xbe\xca\xd5_\xc8\xdd\x14\xc7\x01\x1dO\xc3,\xb9Cz\xc8\x87\xb1&^\xbe\x93eFw\xee\x81\xb7h\x03\x1d\x81\xcb\xc8\xb88\xf8\xe3\xdd\xb1*\xc96\xb5(+l\xb1^\xde\xad\xcc\xb3"\xb7_PK\x07\x08\x07bi\x83\x05\x01\x00\x00\x07\x03\x00\x00PK\x03\x04\x14\x00\x08\x08\x08\x00\x025\xafX\

Download 100.
b'TATAGGATCCATGCAAGTGACTTTATATTATTCTTTAG,DMTON,1325-Bam-F,\r\nTATAGTCGACTTAGTATTTAAGGGCATTGACTTG,DMTON,1325-Sal-R,\r\nTATAGGATCCGTGAGTATGGTTGATTACGAAG,DMTON,DnaB2-Bam-F,\r\nTATAGTCGACTTAGTTACGTTTTTGCAATTCAAAC,DMTON,DnaB2-Sal-R,\r\nAACGGCAAGTTATTTTTCTTCCAACGCTCGTTTAAATAAAT,DMTON,DnaB2-M1,\r\nATTTATTTAAACGAGCGTTGGAAGAAAAATAACTTGCCGTT,DMTON,DnaB2-M2,\r\nAGTGAAGATAGCTAATTCCATAAGAATACTTTAAAAATAAAGTAAAGAATGA,DMT ON,RplJ-trap_F0-1,CUSTOMER\r\nCTTTGGTTATTGCTTTAATGAGTCCTCATTCTTTACTTTATTTTTAAAG,DMT ON,RplJ-trap_F0-2,CUSTOMER\r\nATTAAAGCAATAACCAAAGACCATAACTGGTAAATCACTTGATTTCCTTAATTC,DMT ON,RplJ-trap_F0-3,CUSTOMER\r\nGTTATTAGATATGGTTATCTAACCTACATAACGAATTAAGGAAATCAAGTG,DMT ON,RplJ-trap_F0-4,CUSTOMER\r\nGTTAGATAACCATATCTAATAACAATCTTTGTTATTGTTTTCACTTTACTAATTGAATTTTAAATACAC,DMT ON,RplJ-trap_F0-5,CUSTOMER\r\nGTGTATTTAAAATTCAATTAGTAAAGTGAAAACAAT,DMT ON,RplJ-trap_F0-6,CUSTOMER\r\nAAGTACTTTTTTTGAAATTTCAGATAAGATCTGATAAATCAGATAGGTCGGATTAT,DMT ON,UvrB-trap_F0-1,CUSTOMER\r\nTATGCACAGTTATCTGATAAA

ikchudinov@gmail.com The given mail is valid
viktorvolok94@gmail.com The given mail is valid
i.chudinov@sysbiomed.ru The given mail is valid
volok_vp@sysbiomed.ru The given mail is valid
herr.romanoff@gmail.com The given mail is valid
kuzminajk2000@gmail.com The given mail is valid
t.semashko@gmail.com The given mail is valid


In [ ]:
# !jupyter labextension install @jupyterlab/dataregistry-extension

In [12]:
EXPERIMENT_NAME = "KJE0035"
def test_fucn():
    EXPERIMENT_NAME = google_main()
    dfs_list=process_files(EXPERIMENT_NAME)
    return dfs_list

len(test_fucn())

auth token already exists


HttpAccessTokenRefreshError: invalid_grant: Token has been expired or revoked.

In [15]:
len(dfs_list)

9

In [24]:
dfs_list

[                          0       1     2       3
 3  TTAARTTACAAAAAAACATGGGAC  DMT ON  N-oF  LUKINA
 4   AAAGAATATCTTTTCCTTCAGGG  DMT ON  N-oR  LUKINA
 5    ATGGGACAAGTGAAAATGTCTC  DMT ON  N-iF  LUKINA
 6  AATTACTCATAATCATTTTGACTG  DMT ON  N-iR  LUKINA,
                                                    0       1               2  \
 3  TATACTCGAGTCATTTGTATAGTTCGTCCATTCCGTACAGGAACAG...  DMT ON  dTom_ext_Xho_R   
 4                   ATATCTCGAGTCAGCCATGCCCCAGGAACAGG  DMT ON      dTom_Xho_R   
 5                      CGCGAAATTAATACGACTCACTATAGGGG  DMT ON           T7p_F   
 6                                 GGCCCCAAGGGGTTATGC  DMT ON           T7T_R   
 
               3  
 3  SHCHERBAKOVA  
 4  SHCHERBAKOVA  
 5  SHCHERBAKOVA  
 6  SHCHERBAKOVA  ,
                                                     0        1           2  \
 0                                 GGTGATGGGACGAGCCCAT  DMT OFF  phi29_F1-0   
 1              ATGGGCTCGTCCCATCACCATCACCACCACTCTTCAGG  DMT OFF  phi29_F1-1   
 2  

In [38]:
EXPERIMENT_NAME = "KJE0031"
dfs_list=process_files(EXPERIMENT_NAME)

./requests/KJE0031/праймеры Введенский_10_04_24 - Андрей Введенский.xlsx 6
./requests/KJE0031/праймеры Введенский_10_04_24 - Андрей Введенский.xlsx 4
./requests/KJE0031/2024.04.05_oligos_Alekhina - Olga Alekhina.xlsx 60
./requests/KJE0031/2024.04.05_oligos_Alekhina - Olga Alekhina.xlsx 60
./requests/KJE0031/240415 Chudinov - Ivan Chudinov.xlsx 6
./requests/KJE0031/240415 Chudinov - Ivan Chudinov.xlsx 4
./requests/KJE0031/Дополнительный_рациональный_дизайн - Юлия Кузьмина.xlsx 16
./requests/KJE0031/Дополнительный_рациональный_дизайн - Юлия Кузьмина.xlsx 16
./requests/KJE0031/PAM CAS - Юлия Кузьмина.xlsx 12
./requests/KJE0031/PAM CAS - Юлия Кузьмина.xlsx 12
./requests/KJE0031/ФОРМА ЗАКАЗА - Varvara Shokina.xlsx 28
./requests/KJE0031/ФОРМА ЗАКАЗА - Varvara Shokina.xlsx 26
./requests/KJE0031/Калибровочные буквы - Юлия Кузьмина.xlsx 16
./requests/KJE0031/Калибровочные буквы - Юлия Кузьмина.xlsx 16
./requests/KJE0031/праймеры hMPV - Alexandra Lukina.xlsx 12
./requests/KJE0031/праймеры hMPV -

In [43]:
export_to_input_files(dfs_list, EXPERIMENT_NAME)

In [ ]:
len(dfs_list)

In [47]:
# Использования функции
result = get_emails_data()
response_str = json.dumps(result, indent=2)
unique_emails = remove_duplicate_emails(response_str)
#     print(unique_emails)
unique_emails = emailValid(unique_emails)
#     print(unique_emails)

# write to file
with open(f"./result_input/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_customer_emails.txt", 'w') as f:
    for email in unique_emails:
        f.write(f"{email}\n")

auth token already exists
alekhina.olga@gmail.com The given mail is valid
varvaramys@gmail.com The given mail is valid
v.shokina@sysbiomed.ru The given mail is valid
kuzminajk2000@gmail.com The given mail is valid
ikchudinov@gmail.com The given mail is valid
vvedenskiia@sysbiomed.ru The given mail is valid
vvedenskiia@gmail.com The given mail is valid
lukina.al98@gmail.com  The given mail is invalid
lukina.al98@gmail.com The given mail is valid
herr.romanoff@gmail.com The given mail is valid
